In [1]:
%%html

<style>    
    @import url("css/custom_styles.css")
</style>

<center>
    <h1>
    Transformation Et Manipulation Des Données<br>
    </h1>
    MovieLens - Système de recommendations de films par regroupement<br>
    <br>
    <b>Jean-Francois Gagnon</b><br>
    <br>
    420-A56<br>
    <br>
</center>

# Introduction

<font class="answer">
    
J'ai choisi la base de données [MovieLens Small](https://tinyurl.com/bdhmcfht). Elle se compose de 100836 notes représentant 9742 films. Elle contient également les métadonnées sur 19 genres et des liens sur [TMDB](https://www.themoviedb.org) qui permettront d'augmemter son contenu.
    
Plus spécifiquement, (Voir https://files.grouplens.org/datasets/movielens/ml-latest-small-README.html et décrire en détails le format)
    
    
    
L'objectif de ce projet est de contruire un système de recommentation de films en utilisant les techniques de regroupement vu dans le cours. Les détails du sytème seront abordés plus loins dans ce notebooks

In [2]:
#
# imports utilitaires
#

%matplotlib inline

import json
import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd
import re
import requests
import seaborn as sns
import time

from bs4 import BeautifulSoup
from imblearn.under_sampling import RandomUnderSampler
from sklearn.cluster import MiniBatchKMeans
from sklearn.decomposition import TruncatedSVD

from tqdm.notebook import tqdm

In [3]:
#
# imports faisant partie de nos propres modules
#

import helpers as hlp
import helpers.dataset.MovieLens as mvl

from helpers.jupyter import display_html

#
# fonctions utilitaires
#
def show_na_simple(dataframe):
    na_ = dataframe.isna().sum().to_frame()
    na_.columns = ["NA"]
    display(na_.T)

def show_types_simple(dataframe):
    types_ = dataframe.dtypes.to_frame()
    types_.columns = ["Type"]
    display(types_.T)
    
def split_title(title, regex):
    x = regex.search(title)
    if x:
        title = x.group(1)
        year = x.group(2)
        year = int(year) if year else pd.NA
    else:
        year = pd.NA
    
    return pd.Series({"title": title, "year": year})

def web_scapping_imdb(links_series, config, callback):
    count = links_series.shape[0]
    for index, id in tqdm(links_series.items(), total=count):
        url = "".join([config.scrapping_imdb_url, str(id)])
        with requests.get(url, headers=config.scrapping_imdb_headers.__dict__) as r:
            callback(r, index, id)

            # ne pas surcharger imdb de requetes
            time.sleep(0.05)
    
def json_nested_object(json, keys_iterable):
    for k in keys_iterable:
        if not k in json:
            return None
        json = json[k]
        
    return json

# Prétraitement

<font class="answer">

TODO

In [4]:
#
# parametres configurant nos traitemens
#
configs = hlp.get_configs("config_overrides.json")

#
# obtenir le dataset
#
mvl_dataset = mvl.load(configs.dataset)

In [5]:
#
# explorer links
#
print("Links")

show_na_simple(mvl_dataset.links)
show_types_simple(mvl_dataset.links)
display(mvl_dataset.links.head())

Links


,movieId,imdbId,tmdbId
NA,0,0,8


,movieId,imdbId,tmdbId
Type,int64,string[python],float64


,movieId,imdbId,tmdbId
0,1,0114709,862.0
1,2,0113497,8844.0
2,3,0113228,15602.0
3,4,0114885,31357.0
4,5,0113041,11862.0


<font class="answer">
    
Il manque quelques liens sur [TMDB](https://www.themoviedb.org/) (ce qui explique le type float64). Les liens [IMDB](https://www.imdb.com/) seront par conséquent privélégiés pour fin de web scrapping.

In [6]:
#
# explorer movies
#
print("Movies")

show_na_simple(mvl_dataset.movies)
show_types_simple(mvl_dataset.movies)
display(mvl_dataset.movies.head())

Movies


,movieId,title,genres
NA,0,0,0


,movieId,title,genres
Type,int64,object,object


,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


<font class="answer">

Tel que décris par MovieLens, *title* contient l'année de parution. Nous allons l'extraire afin d'obtenir des caractéritiques "atomiques".

In [7]:
title_year_re = re.compile(configs.dataset.title_regex, flags=0)        
title_year = mvl_dataset.movies.title.apply(split_title, args=(title_year_re,))

# validation de l'extraction
print("Informations extraites")

show_na_simple(title_year)
show_types_simple(title_year)
display(title_year.head())

Informations extraites


,title,year
NA,0,12


,title,year
Type,object,object


,title,year
0,Toy Story,1995
1,Jumanji,1995
2,Grumpier Old Men,1995
3,Waiting to Exhale,1995
4,Father of the Bride Part II,1995


<font class="answer">
   
On peut remarquer qu'il manque quelques années de parution. Il est probablement possible de les fixer en utilisant *links.tmdb* avec le web scrapping.

In [8]:
def imdb_scap_year(response, index, id):
    success = False
    if response.ok:
        bs = BeautifulSoup(response.text, "html.parser")

        imdb_info = bs.find("script", attrs={"type": "application/ld+json"})
        imdb_json = json.loads(imdb_info.text)
        date_published = json_nested_object(imdb_json, ["datePublished"])
        if not success and date_published:
            date_published = pd.to_datetime(date_published)
            year = date_published.year
            success = True

        imdb_info = bs.find("script", attrs={"type": "application/json"})
        imdb_json = json.loads(imdb_info.text)
        releaseYear = json_nested_object(imdb_json, 
                                         ["props", 
                                          "pageProps", 
                                          "mainColumnData", 
                                          "releaseYear",
                                          "year"])
        if not success and releaseYear:
            year = releaseYear
            success = True

    if success:
        title_year.year[index] = year
    else:
        print(url, title_year.title[index], "Failed")

# corriger year via web scrapping avec imdb
year_na = title_year.year.isna()
year_link = mvl_dataset.links.imdbId[ year_na ]

if year_link.shape[0] > 0:
    web_scapping_imdb(year_link, configs.dataset, imdb_scap_year)

# validation du scapping
show_na_simple(title_year)

  0%|          | 0/12 [00:00<?, ?it/s]

,title,year
NA,0,0


In [9]:
#
# mettre a jour movies
# faire attention, ne pas reexecuter ces cellules
# etant donne qu'on modifie le data d'origine
#
mvl_dataset.movies.title = title_year.title
mvl_dataset.movies["year"] = title_year.year.astype(np.int64)

In [10]:
# validation des modalites
print("Unique movieId:", mvl_dataset.movies.movieId.nunique())
print("Unique title:", mvl_dataset.movies.title.nunique())

Unique movieId: 9742
Unique title: 9461


<font class="answer">
    
2 constats:
* *movieId* est effectivement un identifiant tel que décrit par MovieLens. Il sera mis utilisé comme index.
* *title* semble avoir des doublons.
    
*genre* sera manipuler un peu plus loin dans le notebook

# Exploration